In [2]:
def plot_intersection_and_sectionview(df,buildingsData,para):
    
    # create folder
    # =================================================
    if not os.path.exists('data/FigurePlot/intersection' ):
        os.makedirs( 'data/FigurePlot/intersection' )
        print('create directory : intersection' )
    folder_path_intersection = 'data/FigurePlot/intersection/'
    
    if not os.path.exists('data/FigurePlot/sectionview' ):
        os.makedirs( 'data/FigurePlot/sectionview' )
        print('create directory : sectionview' )
    folder_path_sectionview = 'data/FigurePlot/sectionview/'
    
    # =================================================
    user_buffer    = para.Effective_range    # metre
    building_level = para.Level_avg_height   # metre
    rest_building_level = 175 # 過多的樓層數(繪圖調整用)
    
    # =================================================
    for index, row in df.iterrows():
        
        if row['State']=='NLOS' :
        
            user_loc_x = row['geometry'].coords[0][0]
            user_loc_y = row['geometry'].coords[0][1]

            end_loc_x = user_loc_x + user_buffer * math.cos(math.radians(90-row['SAT_AZ']))
            end_loc_y = user_loc_y + user_buffer * math.sin(math.radians(90-row['SAT_AZ']))

            user_point = Point( user_loc_x, user_loc_y, 0 )
            end_point  = Point(  end_loc_x,  end_loc_y, 0 ) 
            user_end_string = LineString([user_point, end_point])

            gdf_string = gpd.GeoDataFrame(columns=['name','geometry'],crs='3857')
            gdf_string['geometry'] = None
            gdf_string.at[1, 'name'] = 'string'
            gdf_string.at[1, 'geometry'] = user_end_string
            gdf_string.at[2, 'name'] = 'user_point'
            gdf_string.at[2, 'geometry'] = user_point
            gdf_string.at[3, 'name'] = 'sat_point'
            gdf_string.at[3, 'geometry'] = end_point
            gdf_string.at[4, 'name'] = 'user_buffer'
            gdf_string.at[4, 'geometry'] = user_point.buffer(user_buffer)

            gdf_buf = gdf_string[gdf_string['name']=='user_buffer']
            gdf_str = gdf_string[gdf_string['name']!='user_buffer']

            buildings_in_buffer =  gpd.sjoin(buildingsData,gdf_buf,how='inner', op='intersects')
            buildings_in_buffer = buildings_in_buffer.drop(['index_right'], axis=1)
            buildings_in_string =  gpd.sjoin(buildings_in_buffer,gdf_str,how='inner', op='intersects')

            # intersection
            # =====================================================================
            fig, ax = plt.subplots(figsize=(40, 25))
            buildingsData.plot(ax=ax)
            # ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)
            buildings_in_buffer.plot(ax=ax,color = 'orange')
            buildings_in_string.plot(ax=ax,color = 'brown')

            gdf_str.plot(aspect=1,ax=ax,color='red')
            gdf_buf.plot(ax=ax,color='green',alpha=0.3)
            
            ax.tick_params(axis='both', which='major', labelsize=20)
            ax.set_xlabel('longitude (metre)', fontsize = 30.0)
            ax.set_ylabel('latitude  (metre)', fontsize = 30.0)
            #ax.set_title( str(row['SAT_name']) + ' ' + row['TIME'].strftime("%Y%m%d %H%M%S") + ' EL_{:.2f} '.format(row['SAT_EL']) + " AZ_{:.2f}".format(row['SAT_AZ']), y=1.08)
            plt.tight_layout()
            plt.savefig(folder_path_intersection + str(row['SAT_name']) + '_' + row['TIME'].strftime("%Y%m%d %H%M%S")+'.png')
            #plt.show()
            plt.close()
            
            # sectionview
            # =====================================================================
            x0 = user_loc_x
            y0 = user_loc_y
            
            
            size_scale = 30
            f_height = (user_buffer/building_level-rest_building_level)/size_scale
            f_width  = (user_buffer/size_scale)
            
            fig, ax = plt.subplots(figsize=(10, 2))
            fig.subplots_adjust(bottom=0.3)
            #fig.set_figheight(f_height+3)
            #fig.set_figwidth(f_width)
            
            ax.set_xlabel('distance (metre)', fontsize = 15.0)
            ax.set_ylabel('building level'  , fontsize = 15.0)
            ax.set_xlim(0, user_buffer) # section view x-axis   
            ax.set_ylim(0, user_buffer/building_level-rest_building_level)    # section view y-axis
            

            ax.plot([0,user_buffer],[0,user_buffer*math.tan( math.radians(row['SAT_EL']))/3.5],c='red')
            
            if (buildings_in_string.empty != True) :
                
                intersection_strings = gdf_string[gdf_string['name']=='string'].intersection(buildings_in_string.unary_union)
                intersection_strings_explode  = intersection_strings.explode(ignore_index=True)
                
                for k in range(0,len(intersection_strings_explode)) :
                    
                    t = intersection_strings_explode[k]
                    #print(t)
                    x1 = t.coords[0][0]
                    y1 = t.coords[0][1]
                    z1 = t.coords[0][2]

                    x2 = t.coords[1][0]
                    y2 = t.coords[1][1]
                    z2 = t.coords[1][2]

                    d1 = ((x1-x0)**2+(y1-y0)**2)**(1/2)
                    d2 = ((x2-x0)**2+(y2-y0)**2)**(1/2)

                    z_max = max(z1,z2)
                    
                    ax.plot([d1,d2],[z_max,z_max],c='blue')   #樓頂橫部
                    ax.plot([d1,d1],[0,z_max],c='blue') # d1 垂直
                    ax.plot([d2,d2],[0,z_max],c='blue') # d2 垂直

                    
                    check_los_d1 = d1*math.tan(math.radians(row['SAT_EL']))/3.5
                    #if(check_los_d1 <= z_max) :
                    #    temp_state = 'NLOS'
                        
                    check_los_d2 = d2*math.tan(math.radians(row['SAT_EL']))/3.5
                    #if(check_los_d2 <= z_max) :
                    #    temp_state = 'NLOS'
                
                #ax.set_title(str(row['SAT_name']) + ' ' + row['TIME'].strftime("%Y%m%d %H%M%S") + ' EL_{:.2f} '.format(row['SAT_EL']))
                plt.savefig(folder_path_sectionview + str(row['SAT_name']) + '_' + row['TIME'].strftime("%Y%m%d %H%M%S")+'.png')
                #plt.show()
                plt.close()